# Box count example
Load basic packages

In [1]:
import box_count_method as bcm
import statistics

import pandas as pd
import pandas_bokeh
from bokeh.io import output_file, output_notebook
from bokeh.models import DatetimeTickFormatter, ranges, Span, Label
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.palettes import linear_palette, Turbo256
pd.set_option('plotting.backend', 'pandas_bokeh')
output_notebook(hide_banner=True)
from IPython.core.display import display, HTML


Change the variable `slice_image` to `False` to open an already prepared folder with multiple image tiles instead of a single large image.

In [2]:
slice_image = True

Select the source file(s):
- folder with the segmented images (by color) 
- large segmented image file (by color) 

The large image will be cut to tiles as defined in `x` and `y` values (standard = 10x10).

Furthermore the image tiles will be processed using the `phaseContent` classdefined in the `box_count_method.py`.

In [3]:
if slice_image:
    phaseContent = bcm.singeFileProcess(x=10, y=10, verbose=True)
else:
    phaseContent = bcm.folderProcess(verbose=True)

Please select the directory with the source image tiles.  12.6465 x 12.6465 nm/px
  The expected sliced images already exists! Doing nothing...
reading CSV "03 segmented_box_count_intermediate.csv"
Loaded 100 images
processing 75 experiments
mean & std deviation of resin/pores
mean & std deviation of C-S-H
mean & std deviation of C₃S
--------------------


basic functions to generate the following diagrams

In [4]:
def get_X_List( values ):
    value_cnt = len(values)
    x_range = list(range(value_cnt))
    x_range.pop(0)
    x_range.append(value_cnt)
    return x_range
    
def getLineDiagram_mean_type1(title, plot_width=1200, legend_location='top_center'):
    p = figure(title=title, plot_width=plot_width, x_range=(1, phaseContent.image_shuffle_count), y_range=(0, 100), plot_height=600)
    p.left[0].formatter.use_scientific = False
    p.xaxis.axis_label = 'image count'
    p.yaxis.axis_label = 'phase content in %'

    stdev_below = 2
    vertical_pos_max = 0
    vertical_pos_min = phaseContent.image_shuffle_count
    colors = linear_palette(Turbo256,len(phaseContent.phase_list))
    for i in range(phaseContent.phase_count):
        key = 'phase_{:02d}_percent'.format(i)
        stdev = phaseContent.stDev_stDev[key]
        mean = phaseContent.stDev_mean[key]
        x_range = get_X_List( stdev )

        positive = list(map(float.__add__, mean, stdev))
        negative = list(map(float.__sub__, mean, stdev))
        
        #phase_names[i]
        p.line(x_range, mean    , line_color=colors[i],                     line_width=2, legend_label=phaseContent.phase_names[i])
        p.line(x_range, positive, line_color=colors[i], line_dash='dashed', line_width=2, legend_label=phaseContent.phase_names[i]+' stdev')
        p.line(x_range, negative, line_color=colors[i], line_dash='dashed', line_width=2)

        for pos in range(len(stdev)):
            if stdev[pos] < stdev_below:
                if vertical_pos_max < pos: vertical_pos_max = pos
                if vertical_pos_min > pos: vertical_pos_min = pos
                break
    
    annotation1 = Span(location=vertical_pos_min, dimension='height', line_color='blue', line_dash='dotted', line_width=1)
    annotation2 = Span(location=vertical_pos_max, dimension='height', line_color='green', line_dash='dotted', line_width=1)
    citation = Label(x=5, y=80, render_mode='css', text='Standard deviation below {} percent after {} to {} images'.format(stdev_below, vertical_pos_min, vertical_pos_max) )
    
    p.add_layout( annotation1 )
    p.add_layout( annotation2 )
    p.add_layout( citation )

    p.legend.location = legend_location
    show(p)

def getLineDiagram_mean_type2(title, plot_width=1200, legend_location='top_center'):
    
    unitArray = [ 'nm', 'µm', 'mm', 'm' ]
    unitFactorArray = [ 1, 1000, 1000000, 1000000000 ]

    tile_area = phaseContent.tile_width*phaseContent.tile_height*phaseContent.scaling['x']**2
    unit = phaseContent.scaling['unit']
    
    unit_pos = -1
    #for unit_item in unitArray:
    #    if unit == unit_item: break
    #    unit_pos += 1
        
    for factor in unitFactorArray:
        if tile_area > factor**2:
            unit_pos += 1

    unit = unitArray[unit_pos]
    tile_area = tile_area / (unitFactorArray[unit_pos]**2)

    #print('{} {}'.format(tile_area, unit))

    p = figure(title=title, plot_width=plot_width, x_range=(tile_area, phaseContent.image_shuffle_count * tile_area), y_range=(0, 100), plot_height=600)
    p.left[0].formatter.use_scientific = False

    p.xaxis.axis_label = 'area in {}²'.format( unit )
    p.yaxis.axis_label = 'phase content in %'

    stdev_below = 2
    vertical_pos_max = 0
    vertical_pos_min = phaseContent.image_shuffle_count
    colors = linear_palette(Turbo256,len(phaseContent.phase_list))
    for i in range(phaseContent.phase_count):
        key = 'phase_{:02d}_percent'.format(i)
        stdev = phaseContent.stDev_stDev[key]
        mean = phaseContent.stDev_mean[key]
        x_range = get_X_List( stdev )
        x_range = list( map(lambda x: x * tile_area, x_range) )

        positive = list(map(float.__add__, mean, stdev))
        negative = list(map(float.__sub__, mean, stdev))
        
        #phase_names[i]
        p.line(x_range, mean    , line_color=colors[i],                     line_width=2, legend_label=phaseContent.phase_names[i])
        p.line(x_range, positive, line_color=colors[i], line_dash='dashed', line_width=2, legend_label=phaseContent.phase_names[i]+' stdev')
        p.line(x_range, negative, line_color=colors[i], line_dash='dashed', line_width=2)

        for pos in range(len(stdev)):
            if stdev[pos] < stdev_below:
                if vertical_pos_max < pos: vertical_pos_max = pos
                if vertical_pos_min > pos: vertical_pos_min = pos
                break
    
    annotation1 = Span(location=vertical_pos_min*tile_area, dimension='height', line_color='blue', line_dash='dotted', line_width=1)
    annotation2 = Span(location=vertical_pos_max*tile_area, dimension='height', line_color='green', line_dash='dotted', line_width=1)
    citation = Label(x=5*phaseContent.scaling['x'], y=80, render_mode='css', text='Standard deviation below {} percent after {} to {} images'.format(stdev_below, vertical_pos_min, vertical_pos_max) )
    
    p.add_layout( annotation1 )
    p.add_layout( annotation2 )
    p.add_layout( citation )

    p.legend.location = legend_location
    show(p)

generate Diagrams 

In [5]:
getLineDiagram_mean_type1('{} repeats per image count'.format( phaseContent.repeat_sampling ))

In [6]:
getLineDiagram_mean_type2('{} repeats per image count'.format( phaseContent.repeat_sampling ))

In [7]:
repeats = 100
phaseContent.reprocess_mean_and_stdev(repeats, verbose=False)
getLineDiagram_mean_type1('{} repeats per image count'.format( repeats ))

In [8]:
repeats = 200
phaseContent.reprocess_mean_and_stdev(repeats, verbose=False)
getLineDiagram_mean_type1('{} repeats per image count'.format( repeats ))

In [9]:
print(phaseContent.scaling)

{'x': 12.646541803143931, 'y': 12.646541803143931, 'unit': 'nm', 'editor': 'ImageJ 1.53c'}
